In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats
from astropy.io import fits
from astropy.stats import sigma_clip
from astropy.coordinates import SkyCoord

In [ ]:
# reset defalult plotting values
plt.rcParams['figure.figsize'] = (10, 7)
plt.rc('font', family='sans-serif')
plt.rc('axes', labelsize=14)
plt.rc('axes', labelweight='bold')
plt.rc('axes', titlesize=16)
plt.rc('axes', titleweight='bold')
plt.rc('axes', linewidth=2)

# Photometry
## quantifying light from astronomical sources

<img src="media/photometry.png" width=400>

### Prof. Robert Quimby
&copy; 2019 Robert Quimby

## In this tutorial you will...

- Learn what it means to do aperture photometry on a celestial target
- measure the number of object counts above the background level recorded by a CCD
- determine the uncertainty in this measurement
- use the `astropy` package, `photutils` to measure counts in a circular aperture

## Terminology

![](http://i-fiberoptics.com/images/if-pm.jpg)

In the context of photometry, "aperture" refers **not** to the size of your telescope, but to the area of the telescope's focal plane used when measuring light.

## Simplest form: box aperture photometry

In [ ]:
# Load (processed) image data
image = fits.getdata('media/example.fits')

In [ ]:
# show a close-up view of a star
x, y = 244, 640
margin = 10

# show the stamp
stamp = image[y - margin : y + margin + 1, x - margin : x + margin + 1]
xorigin, yorigin = x - margin, y - margin # for later use
extent = [x - margin,  x + margin + 1, y - margin, y + margin + 1]
plt.imshow(stamp, origin='lower', cmap='gray', vmin=1350, vmax=1700, extent=extent);

In [ ]:
# define a background aperture
bgx, bgy = x, y + 20
bgstamp = image[bgy - margin : bgy + margin + 1, bgx - margin : bgx + margin + 1]
bgextent = [bgx - margin,  bgx + margin + 1, bgy - margin, bgy + margin + 1]
plt.imshow(bgstamp, origin='lower', cmap='gray', vmin=1350, vmax=1700, extent=bgextent);

In [ ]:
# total counts in the target aperture
counts = ????

# total counts in the background aperture
bgcounts = ????

print("    target aperture has {:.0f} counts".format(counts))
print("background aperture has {:.0f} counts".format(bgcounts))

In [ ]:
# how many counts do we get from just the target?
print("That is {:.0f} counts from the object".format(????))

## What is the Uncertainty in the Object Counts?

#### Recall Propagation of Uncertainties

* Given a function $f$ of several variables, $x_0, x_1 ... x_{n-1}$, if the variables have random, independent uncertainties, $\sigma_{x_0}, \sigma_{x_1} ... \sigma_{x_{n-1}}$, then the uncertainty in the function, $\sigma_f$, is:

$$ \sigma_f = \sqrt{ \left( {\partial f \over \partial x_0 }\sigma_{x_0}\right)^2
                   + \left( {\partial f \over \partial x_1 }\sigma_{x_1}\right)^2
                   + ...
                   + \left( {\partial f \over \partial x_n }\sigma_{x_{n-1}}\right)^2
                    } $$

The object counts, $f$, are computed from the equation:
$$f = \sum_{i=0}^{N-1} (S_i + B_i - \bar{B})$$
where:
 * $N$ is the number of pixels (i.e. the area) in the aperture
 * $S_i$ is the source (star) counts in the $i^{\rm th}$ pixel,
 * $B_i$ is the background counts in the $i^{\rm th}$ pixel, 
 * $\bar{B}$ is the average value of the $M$ pixels in the background sample
 
$$\bar{B} = {1 \over M}\sum_{j=0}^{M-1} B_j$$

Estimate the uncertainty in $\bar{B}$ from the background sample.

Assume that the total source counts, $S = \sum S_i$, are drawn from a Poisson distribution. Then using the fact that Poisson distributions have $\sigma^2 = \mu$, we find:

$$\sigma^2_S = S$$

So the uncertainty in the measured object counts, $\sigma_f$ is given by:
$$\sigma_f = \sqrt{ S + N\sigma_B^2 + {N^2\sigma_B^2 \over M} } $$

## Calculate the uncertainty in the counts measurement

In [ ]:
# photometric uncertainty
obj_var = ????
bg_var = stamp.size * bgstamp.var()
avbg_var = bgstamp.std()**2 / bgstamp.size * stamp.size**2
ecounts = np.sqrt( obj_var + bg_var +  avbg_var)
print("Total uncertainty is {:.0f} counts".format(ecounts))
print("({:.0f} object, {:.0f} background, {:.0f} background average)".format(obj_var**0.5, bg_var**0.5, avbg_var**0.5))

## Background pixels just add noise

In [ ]:
def plot_example_star_image(showaperture=False):
    star = fits.getdata('media/star_image.fits')
    plt.figure(figsize=(10, 10))
    plt.imshow(star, origin='lower')
    ny, nx = star.shape
    for x in range(nx):
        for y in range(ny):
            plt.annotate("{:.0f}".format(star[y, x]), xy=(x, y), va='bottom', ha='center')
            
    if showaperture:
        cx, cy = 5.2, 5.6
        circle= plt.Circle((cx, cy), radius=3, fill=False, color='r', lw=2)
        ax=plt.gca()
        ax.add_patch(circle);

In [ ]:
plot_example_star_image()

## Circular Aperture Photometry

- sum up the light from a star in a circular aperture

In [ ]:
plot_example_star_image(showaperture=True)

## Decide where to center the aperture

In [ ]:
# get the x,y coordinates for each pixel in the stamp
yinds, xinds = np.indices(stamp.shape)

# calculate the "weight" for each pixel
weight = ????

## calculate the "center of mass" in each coordinate
cmx = ????
cmy = ????
print("object is centered at {:.2f}, {:.2f}".format(cmx, cmy))

In [ ]:
# mark the intensity weighted center on the image stamp
plt.imshow(stamp, origin='lower', cmap='gray', vmin=1350, vmax=1700)
plt.plot(cmx, cmy, 'r+', ms=14);

## Decide on the radius for the aperture

Light is spread out due to refracton and atomspheric seeing
- cannot measure 100% of the light in a finite radius aperture
- but you can measure as close to this as you care to

In [ ]:
# plot the object's profile along one axis
profile = ????
profile -= ????
profile /= ????
plt.plot(profile)
plt.axvline(cmx, color='r', ls='dashed')
plt.grid();

In [ ]:
plt.plot(profile)
plt.axvline(cmx, color='r', ls='dashed')
fwhm = ????
plt.plot([cmx - fwhm/2, cmx + fwhm/2], [0.5, 0.5], color='red', ls='dotted')

plt.xlabel('Stamp Column Number')
plt.ylabel('Column Sum');
plt.grid()

In [ ]:
# show the circular aperture on the image stamp
plt.imshow(stamp, origin='lower', cmap='gray', vmin=1350, vmax=5700)
plt.plot(cmx, cmy, 'r+', ms=14);

circle = plt.Circle((cmx, cmy), radius=fwhm, fill=False, color='r', lw=2)
ax = plt.gca()
ax.add_patch(circle);

## Sum up the Counts in a Circular Aperture

In [ ]:
plot_example_star_image(showaperture=True)

Rules for measuring counts in an aperture
* if a pixel is wholly contained in the aperture, add all of its counts
* if $XX$% of a pixel is inside the aperture, add $XX$% of the counts

## Use the `photutils` package for the dirty work

In [ ]:
# sum the counts in the aperture
import photutils
aperture = photutils.CircularAperture( ???? )
phot_table = photutils.aperture_photometry(????)

In [ ]:
# look at the photometry table
phot_table

## Define a background sample

In [ ]:
# show the circular aperture on the image stamp
cmx2, cmy2 = ????

plt.imshow(image, origin='lower', cmap='gray', vmin=1350, vmax=1700)
plt.xlim(cmx2 - 7 * fwhm, cmx2 + 7 * fwhm)
plt.ylim(cmy2 - 7 * fwhm, cmy2 + 7 * fwhm)

plt.plot(cmx2, cmy2, 'r+', ms=14);
circle = plt.Circle((cmx2, cmy2), radius=fwhm, fill=False, color='r', lw=2)
ax = plt.gca()
ax.add_patch(circle);

## Get the background sample pixels in an annulus around the target position

In [ ]:
# calculate the distance from each image pixel to the centroid
imy, imx = ????
dists = ????
wbg = ????
bg_sample = ????

In [ ]:
# measure the background subtracted counts in the aperture
phot_table = photutils.aperture_photometry(????)
print("{:.0f} counts from the object".format(????))

## Determine the Uncetainty in the Circular Aperture Counts

In [ ]:
obj_var = ????
bg_var = ???? * bg_sample.var()
avbg_var = bg_sample.var() / bg_sample.size * ????**2
ecounts = np.sqrt(obj_var + bg_var +  avbg_var)
print("Total uncertainty is {:.0f} counts".format(ecounts))
print("({:.0f} object, {:.0f} background, {:.0f} background average)".format(obj_var**0.5, bg_var**0.5, avbg_var**0.5))

## More on photometry

[`photutils.aperture` photometry documentation](https://photutils.readthedocs.io/en/stable/aperture.html)